In [2]:
# Import Dependencies
import json
import requests
import gmaps
from config import api_key
from config import gkey
from pprint import pprint
import pandas as pd
from datetime import date
from datetime import timedelta
import requests
import urllib
from urllib.request import urlopen
import json
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import geopy.distance as gp
import warnings
warnings.filterwarnings('ignore')

# Food Deserts

A Food Desert is an area that has limited access to affordable and nutritious food. Food deserts tend to be inhabited by low-income residents with reduced mobility; this makes them a less attractive market for large supermarket chains. Food deserts lack suppliers of fresh foods, such as meats, fruits, and vegetables. Instead, the available foods are often processed and high in sugar and fats, which are known contributors to the proliferation of obesity in the U.S.

### Beginning the data-gathering process

We found that the Percentage of Free or Reduced Lunches (Percent_FRL) at school is a strong indicator of poverty in an area:
https://nces.ed.gov/blogs/nces/post/free-or-reduced-price-lunch-a-proxy-for-poverty

Using this metric, we identified 90 schools across 7 counties of Georgia with Percent_FRL of >= 92%. We collected these school names, physical addresses and the their Percent_FRL in a CSV file.

### Approach
1. Get the name and addresses of all the schools which have the >90% FRL (free-reduced lunch) (%FRL is used as an indicator of poverty).
2. Find out the number of grocery stores within 5-10 miles of these schools (usually people live near the schools and higher percentage FRL means that the neighborhood is poor).
3. Create a Heat Map these grocery stores.
4. Find out any Fast Food joints / Dollar Stores within 5 miles of these locations. Usually in Food Deserts there is proliferation of these establishments selling processed foods.
5. Creat a Heat Map these Fast Food places / Dollar Stores.

In [20]:
infile = pd.read_csv("../Datasets/GA_County_Data.csv")
county_df = pd.DataFrame(infile)
county_df

,County_ID,County_Name,Per_Capita_Income,Percent_FRL,Latitude,Longitude
0,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206
1,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362
2,633,Cobb County,"63,214.00",39.14,33.962944,-84.544548
3,634,Coffee County,"35,507.00",87.34,31.508705,-82.850724
4,635,Colquitt County,"37,418.00",86.21,31.190741,-83.789843
5,644,DeKalb County,"56,428.00",68.56,33.774350,-84.297124
6,650,Echols County,"31,945.00",91.26,30.703350,-83.027601
7,658,Forsyth County,"67,859.00",13.68,34.206598,-84.138697
8,660,Fulton County,"95,683.00",42.17,33.758231,-84.391101
9,667,Gwinnett County,"46,886.00",50.30,33.966996,-83.994210


## Getting Coordinates of the Counties

In [63]:
stores = []
errors = []
for row in county_df.itertuples():
    print(row.County_Name)
    County_ID = row.County_ID
    County_Name = row.County_Name
    County_Income = row.Per_Capita_Income
    Poverty_Rate = row.Percent_FRL  
    County_Lat = row.Latitude
    County_Long = row.Longitude 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params = {"key": gkey, 
          "location": f"{County_Lat},{County_Long}", 
          "radius": 50000, 
          "type": "supermarket"}

    data = requests.get(base_url, params = params).json()
    
#     print(len(data))
#     pprint(data)
    try:
        if data['status'] == 'OK':
            i = 0
            for r in data:
                Store_Lat = data['results'][i]['geometry']['location']['lat']
                Store_Long = data['results'][i]['geometry']['location']['lng']
                Store_Name = data['results'][i]['name']
                Store_Address = data['results'][i]['vicinity']
                Store_PlaceID = data['results'][i]['place_id']
                coords_1 = (County_Lat, County_Long)
                coords_2 = (Store_Lat, Store_Long)
                Dist = (gp.distance(coords_1, coords_2).km) * 0.621371
#                 print(geopy.distance.distance(coords_1, coords_2).km) 0.621371
                current_record[i] = [County_ID, County_Name, County_Income, Poverty_Rate, County_Lat, County_Long,\
                                    Store_Name, Store_Lat, Store_Long, Store_Address, Dist]
                stores.append(current_record[i])
                i = i + 1
#                 print(current_record[i])
    except IndexError:
        errors.append('Index Out of Range')    
    


store_df = pd.DataFrame(stores, columns=['County_ID', 'County_Name', 'County_Income', 'Poverty_Rate', 'County_Lat', 'County_Long', 'Store_Name',\
            'Store_Lat','Store_Long', 'Store_Address','Dist'])
store_df


Baldwin County
Clayton County
Cobb County
Coffee County
Colquitt County
DeKalb County
Echols County
Forsyth County
Fulton County
Gwinnett County
Johnson County
Lowndes County
Meriwether County
Montgomery County
Washington County
Wilkes County


,County_ID,County_Name,County_Income,Poverty_Rate,County_Lat,County_Long,Store_Name,Store_Lat,Store_Long,Store_Address,Dist
0,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Piggly Wiggly,32.883275,-83.333284,"123 South Main Street, Gordon",14.959419
1,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Ingles Market,33.003710,-83.545153,"261 West Clinton Street, Gray",19.160490
2,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",0.688231
3,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",1.949584
4,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362,Whole Foods Market,33.775234,-84.365829,"650 Ponce De Leon Avenue Northeast, Atlanta",18.561028
5,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362,Trader Joe's,33.779655,-84.367910,"931 Monroe Drive Northeast, Atlanta",18.869139
6,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362,Publix Super Market at The Plaza Midtown,33.780480,-84.388809,"950 West Peachtree Street Northwest, Atlanta",19.002858
7,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362,The Fresh Market,33.811965,-84.391409,"2099 Peachtree Road Northwest, Atlanta",21.177290
8,633,Cobb County,"63,214.00",39.14,33.962944,-84.544548,Whole Foods Market,33.775234,-84.365829,"650 Ponce De Leon Avenue Northeast, Atlanta",16.521371
9,633,Cobb County,"63,214.00",39.14,33.962944,-84.544548,Trader Joe's,33.779655,-84.367910,"931 Monroe Drive Northeast, Atlanta",16.208438


In [64]:
print(len(store_df))

59


In [65]:
print(errors)

['Index Out of Range']


In [66]:
grocery_stores_df = store_df.loc[(store_df["Store_Name"].str.contains(r'^(?:(?!Dollar).)*$'))]
grocery_stores_df

,County_ID,County_Name,County_Income,Poverty_Rate,County_Lat,County_Long,Store_Name,Store_Lat,Store_Long,Store_Address,Dist
0,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Piggly Wiggly,32.883275,-83.333284,"123 South Main Street, Gordon",14.959419
1,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Ingles Market,33.003710,-83.545153,"261 West Clinton Street, Gray",19.160490
2,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Piggly Wiggly,33.071728,-83.225574,"731 South Wayne Street, Milledgeville",0.688231
3,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Food Depot,33.101961,-83.251373,"1850 North Columbia Street, Milledgeville",1.949584
4,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362,Whole Foods Market,33.775234,-84.365829,"650 Ponce De Leon Avenue Northeast, Atlanta",18.561028
5,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362,Trader Joe's,33.779655,-84.367910,"931 Monroe Drive Northeast, Atlanta",18.869139
6,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362,Publix Super Market at The Plaza Midtown,33.780480,-84.388809,"950 West Peachtree Street Northwest, Atlanta",19.002858
7,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362,The Fresh Market,33.811965,-84.391409,"2099 Peachtree Road Northwest, Atlanta",21.177290
8,633,Cobb County,"63,214.00",39.14,33.962944,-84.544548,Whole Foods Market,33.775234,-84.365829,"650 Ponce De Leon Avenue Northeast, Atlanta",16.521371
9,633,Cobb County,"63,214.00",39.14,33.962944,-84.544548,Trader Joe's,33.779655,-84.367910,"931 Monroe Drive Northeast, Atlanta",16.208438


In [67]:
dollar_stores_df = store_df.loc[(store_df["Store_Name"].str.contains('Dollar'))]
dollar_stores_df

,County_ID,County_Name,County_Income,Poverty_Rate,County_Lat,County_Long,Store_Name,Store_Lat,Store_Long,Store_Address,Dist
13,634,Coffee County,"35,507.00",87.34,31.508705,-82.850724,Family Dollar,31.306978,-82.849332,"652 North Main Street, Pearson",13.898342
26,650,Echols County,"31,945.00",91.26,30.703350,-83.027601,Family Dollar,30.836751,-83.317776,"964 Saint Augustine Road, Valdosta",19.555092
37,683,Johnson County,"27,965.00",94.38,32.730663,-82.719275,Family Dollar,32.526701,-82.914105,"1206 Telfair Street, Dublin",18.072144
38,683,Johnson County,"27,965.00",94.38,32.730663,-82.719275,Family Dollar,32.732101,-82.721858,"212 West Trilby Street, Wrightsville",0.180186
39,683,Johnson County,"27,965.00",94.38,32.730663,-82.719275,Family Dollar,32.978675,-82.810240,"309 South Harris Street, Sandersville",17.891043
42,692,Lowndes County,"41,117.00",43.76,30.841890,-83.278880,Family Dollar,30.836751,-83.317776,"964 Saint Augustine Road, Valdosta",2.339034
43,692,Lowndes County,"41,117.00",43.76,30.841890,-83.278880,Family Dollar,30.682154,-83.216426,"1006 Lakes Boulevard, Lake Park",11.614313
44,692,Lowndes County,"41,117.00",43.76,30.841890,-83.278880,Family Dollar,30.856150,-83.280243,"1905 North Ashley Street, Valdosta",0.985686
47,699,Meriwether County,"39,121.00",88.02,33.029043,-84.713045,Family Dollar,33.278260,-85.097761,"291 Court Square, Franklin",28.148083
50,703,Montgomery County,"34,061.00",94.83,32.174174,-82.594652,Family Dollar,32.211571,-82.401637,"1101h East 1st Street, Vidalia",11.598979


In [68]:
len(dollar_stores_df)

13

In [69]:
fastfoodplace = []
errors = []
for row in county_df.itertuples():
    
    County_ID = row.County_ID
    County_Name = row.County_Name
    County_Income = row.Per_Capita_Income
    Poverty_Rate = row.Percent_FRL  
    County_Lat = row.Latitude
    County_Long = row.Longitude 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params = {"key": gkey, 
          "location": f"{County_Lat},{County_Long}", 
          "radius": 50000, 
          "type": "restaurant"}

    data = requests.get(base_url, params = params).json()
    
    try:
        if data['status'] == 'OK':
            i = 0
            for r in data:
                Store_Lat = data['results'][i]['geometry']['location']['lat']
                Store_Long = data['results'][i]['geometry']['location']['lng']
                Store_Name = data['results'][i]['name']
                Store_Address = data['results'][i]['vicinity']
                Store_PlaceID = data['results'][i]['place_id']
                coords_1 = (County_Lat, County_Long)
                coords_2 = (Store_Lat, Store_Long)
                Dist = (gp.distance(coords_1, coords_2).km) * 0.621371

                current_record[i] = [County_ID, County_Name, County_Income, Poverty_Rate, County_Lat, County_Long,\
                                    Store_Name, Store_Lat, Store_Long, Store_Address, Dist]
                fastfoodplace.append(current_record[i])
    
                i = i + 1

    except IndexError:
        errors.append('Index Out of Range')    
    


fastfoodplace_df = pd.DataFrame(fastfoodplace, columns=['County_ID', 'County_Name', 'County_Income', 'Poverty_Rate', 'County_Lat', 'County_Long', 'Store_Name',\
            'Store_Lat','Store_Long', 'Store_Address','Dist'])

fastfoodplace_df


,County_ID,County_Name,County_Income,Poverty_Rate,County_Lat,County_Long,Store_Name,Store_Lat,Store_Long,Store_Address,Dist
0,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Linger Longer Steakhouse,33.439587,-83.197479,"1 Lake Oconee Trail, Greensboro",24.744167
1,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Gaby's by the Lake,33.438713,-83.192434,"1 Lake Oconee Trail, Greensboro",24.706855
2,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Ruby Tuesday,33.110224,-83.256479,"2440 North Columbia Street, Milledgeville",2.572405
3,605,Baldwin County,"36,399.00",92.88,33.081466,-83.228206,Subway,33.295486,-83.388971,"201 Walmart Drive, Eatonton",17.444697
4,631,Clayton County,"32,702.00",92.55,33.506014,-84.357362,Hyatt Regency Atlanta,33.761375,-84.387356,"265 Peachtree Street Northeast, Atlanta",17.684217
...,...,...,...,...,...,...,...,...,...,...,...
59,750,Washington County,"39,151.00",85.62,32.983578,-82.811602,Subway,33.006651,-82.396953,"114 U.S. 1, Louisville",24.132034
60,757,Wilkes County,"42,093.00",94.17,33.746467,-82.740314,Subway,33.462135,-82.506406,"318 West Hill Street, Thomson",23.790120
61,757,Wilkes County,"42,093.00",94.17,33.746467,-82.740314,Subway,33.883483,-83.122709,"726 Athens Highway, Lexington",23.940145
62,757,Wilkes County,"42,093.00",94.17,33.746467,-82.740314,Subway,33.910101,-82.294075,"201 South Mine Street, McCormick",28.036256


In [70]:
len(fastfoodplace_df)

64

In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Store</dt><dd>{Store_Name}</dd>
<dt>County</dt><dd>{County_Name}</dd>
</dl>
"""
# <dt>School</dt><dd>{School_Name}</dd>

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
grocery_store_info = [info_box_template.format(**row) for index, row in grocery_stores_df.iterrows()]
locations = grocery_stores_df[["Store_Lat","Store_Long"]]

In [72]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = grocery_store_info)

fig.add_layer(marker_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# coord = dollar_stores1_df[["Latitude","Longitude"]]
# poverty = dollar_stores1_df["Humidity"]

In [ ]:
# # Plot Heatmap
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure()

# # Create heat layer
# heat_layer = gmaps.heatmap_layer(coord, weights=humidity, dissipating=False, max_intensity=50, point_radius=1)


# # Add layer
# fig.add_layer(heat_layer)

# # Display figure
# fig

In [73]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Store</dt><dd>{Store_Name}</dd>
<dt>County</dt><dd>{County_Name}</dd>
</dl>
"""
# <dt>School</dt><dd>{School_Name}</dd>

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
dollar_store_info = [info_box_template.format(**row) for index, row in dollar_stores_df.iterrows()]
locations = dollar_stores_df[["Store_Lat","Store_Long"]]

In [74]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = dollar_store_info)

fig.add_layer(marker_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

In [75]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Store</dt><dd>{Store_Name}</dd>
<dt>County</dt><dd>{County_Name}</dd>
</dl>
"""
# <dt>School</dt><dd>{School_Name}</dd>

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
fastfood_info = [info_box_template.format(**row) for index, row in fastfoodplace_df.iterrows()]
locations = fastfoodplace_df[["Store_Lat","Store_Long"]]

In [44]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = fastfood_info)

fig.add_layer(marker_layer)

# Display figure

fig


Figure(layout=FigureLayout(height='420px'))

In [76]:
output_grocery_store_file = "../Output/county_grocery_store_final.csv"
grocery_stores_df.to_csv(output_grocery_store_file, sep=',', index=False)

In [77]:
output_dollar_store_file = "../Output/county_dollar_store_final.csv"
dollar_stores_df.to_csv(output_dollar_store_file, sep=',', index=False)

In [78]:
output_fast_food_file = "../Output/county_fast_food_final.csv"
fastfoodplace_df.to_csv(output_fast_food_file, sep=',', index=False)

In [79]:
fastfoodplace_df.to_json('../Basic/resources/county_fast_food_final.json', orient = 'records', compression = 'infer')

In [80]:
grocery_stores_df.to_json('../Basic/resources/county_grocery_stores_final.json', orient = 'records', compression = 'infer')

In [81]:
dollar_stores_df.to_json('../Basic/resources/county_dollar_stores_final.json', orient = 'records', compression = 'infer')